In [1]:
# uncomment the code below to install active_learning_ratio_estimation from source
# if you do not do this, unless you already have it installed in this environment, this notebook will not run

import os, sys, tempfile
original_dir = os.getcwd()
REPO_NAME = 'active_learning_ratio_estimation'
BRANCH = 'master'
tempdir = tempfile.gettempdir()
os.chdir(tempdir)
os.system(f'git clone --single-branch --branch {BRANCH} https://github.com/cjs220/{REPO_NAME}.git')
sys.path.insert(0, os.path.join(tempdir, REPO_NAME))
os.chdir(original_dir)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
from sklearn import clone
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.datasets import make_sparse_spd_matrix

from active_learning_ratio_estimation.model import SinglyParameterizedRatioModel, DenseClassifier, FlipoutClassifier
from active_learning_ratio_estimation.dataset import ParamGrid, SinglyParameterizedRatioDataset

# get_ipython().run_line_magic('matplotlib', 'inline')

np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
class MultiDimToyModel(tfd.TransformedDistribution):

    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta

        # compose linear transform
        R = make_sparse_spd_matrix(5, alpha=0.5, random_state=7).astype(np.float32)
        self.R = R
        transform = tf.linalg.LinearOperatorFullMatrix(R)
        bijector = tfp.bijectors.AffineLinearOperator(scale=transform)

        super().__init__(distribution=self.z_distribution, bijector=bijector)

    @property
    def z_distribution(self):
        z_distribution = tfd.Blockwise([
            tfd.Normal(loc=self.alpha, scale=1),  # z1
            tfd.Normal(loc=self.beta, scale=3),  # z2
            tfd.MixtureSameFamily(
                mixture_distribution=tfd.Categorical(probs=[0.5, 0.5]),
                components_distribution=tfd.Normal(
                    loc=[-2, 2],
                    scale=[1, 0.5]
                )
            ),  # z3
            tfd.Exponential(3),  # z4
            tfd.Exponential(0.5),  # z5
        ])
        return z_distribution

In [ ]:
# Plot histograms / correlations of true distributions
true_alpha = 1
true_beta = -1
p_true = MultiDimToyModel(alpha=1, beta=-1)
X_true = p_true.sample(500)
# fig = corner(X_true, bins=20, smooth=0.85, labels=["X0", "X1", "X2", "X3", "X4"])

In [ ]:
# find exact maximum likelihood
var_alpha = tf.Variable(tf.constant(0, dtype=tf.float32))
var_beta = tf.Variable(tf.constant(0, dtype=tf.float32))
p_var = MultiDimToyModel(var_alpha, var_beta)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
n_iter = int(1e3)
nll = tf.function(lambda: -tf.keras.backend.sum(p_var.log_prob(X_true)))

for i in range(n_iter):
    optimizer.minimize(nll, [var_alpha, var_beta])

alpha_mle = var_alpha.numpy()
beta_mle = var_beta.numpy()
theta_mle = np.array([alpha_mle, beta_mle])
max_log_prob = p_var.log_prob(X_true)

print(f'Exact MLE: alpha={alpha_mle}, beta={beta_mle}')

In [ ]:
# create dataset for fitting
param_grid_train = ParamGrid(bounds=[(-3, 3), (-3, 3)], num=30)
theta_0 = np.array([alpha_mle, beta_mle])
ds = SinglyParameterizedRatioDataset(
    simulator_func=MultiDimToyModel,
    theta_0=theta_0,
    theta_1_iterator=param_grid_train,
    n_samples_per_theta=500
)

In [ ]:
# hyperparams
epochs = 1
patience = 2
validation_split = 0.1
n_hidden = (40, 40)

# create model
bayesian_estimator = FlipoutClassifier(n_hidden=n_hidden, activation='relu',
                                       epochs=epochs, patience=patience,
                                       validation_split=validation_split)
model = SinglyParameterizedRatioModel(estimator=bayesian_estimator, calibration_method=None)

In [ ]:
def fit_predict(clf, grid):
    clf.fit(ds)
    nllr_pred = clf.nllr_param_scan(x=X_true, param_grid=grid)
    return nllr_pred

In [ ]:
num_plot = 50
alpha_bounds = (0.75, 1.25)
beta_bounds = (-2, 0)
plot_grid = ParamGrid(bounds=[alpha_bounds, beta_bounds], num=num_plot)
Alphas, Betas = plot_grid.meshgrid()

# fit model and predict contours
pred_contours = fit_predict(model, grid=plot_grid)

In [ ]:
# Calculate contours of exact negative log likelihood ratio

@tf.function
def nllr_exact(alpha, beta, X):
    p_theta = MultiDimToyModel(alpha=alpha, beta=beta)
    return -tf.keras.backend.sum((p_theta.log_prob(X) - max_log_prob))


exact_contours = np.zeros_like(Alphas)
for i in range(num_plot):
    for j in range(num_plot):
        alpha = tf.constant(Alphas[i, j])
        beta = tf.constant(Betas[i, j])
        nllr = nllr_exact(alpha, beta, X_true)
        exact_contours[i, j] = nllr

In [ ]:
# Plot exact and predicted contours


def plot_contours(contours, ax):
    ax.contour(*plot_grid.meshgrid(), 2 * contours, levels=[chi2.ppf(0.683, df=2),
                                                            chi2.ppf(0.9545, df=2),
                                                            chi2.ppf(0.9973, df=2)], colors=["w"])
    ax.contourf(*plot_grid.meshgrid(), 2 * contours, vmin=0, vmax=30)
    ax.plot([true_alpha], [true_beta], "ro", markersize=8, label='True')
    ax.plot([alpha_mle], [beta_mle], "go", markersize=8, label='MLE')
    ax.set_xlim(*alpha_bounds)
    ax.set_ylim(*beta_bounds)
    ax.set_xlabel(r"$\alpha$")
    ax.set_ylabel(r"$\beta$")


f, axarr = plt.subplots(2, sharex=True)
for i, contours in enumerate([exact_contours, pred_contours]):
    plot_contours(contours, ax=axarr[i])

axarr[0].legend(loc='upper center', fancybox=True, shadow=True)
contour_mae = np.abs(pred_contours - exact_contours).mean()
print('Contour MAE: ', contour_mae)
plt.savefig('contours.pdf')